In [5]:
import numpy
import os

# Training Data

image_path = 'numpy_data/Ultrasound Volumes/'
segmentation_path = 'numpy_data/Patient Segmentations/'
image_file_list = [image_path + f for f in os.listdir(image_path) if f.endswith('.npy')]
segmentation_file_list = [segmentation_path + f for f in os.listdir(segmentation_path) if f.endswith('.npy')]

image_arrays = []
for file in image_file_list:
    image_array = numpy.load(file)
    image_array.resize(16, 256, 256)
    image_array = numpy.expand_dims(image_array, 3)
    image_arrays.append(image_array)

segmentation_arrays = []
for file in segmentation_file_list:
    segmentation_array = numpy.load(file)
    segmentation_array.resize(16, 256, 256)
    segmentation_array = numpy.expand_dims(segmentation_array, 3)
    segmentation_arrays.append(segmentation_array)

stacked_image_array = numpy.stack(image_arrays)
stacked_segmentation_array = numpy.stack(segmentation_arrays)

print(stacked_image_array.shape)
print(stacked_segmentation_array.shape)

# export as numpy files
image_file = 'numpy_data/Stacked Arrays/stacked_image_array'
segmentation_file = 'numpy_data/Stacked Arrays/stacked_segmentation_array'

numpy.save(image_file, stacked_image_array)
numpy.save(segmentation_file, stacked_segmentation_array)

(23, 16, 256, 256, 1)
(23, 16, 256, 256, 1)


In [6]:
# Testing Data

image_path = 'numpy_data/Ultrasound Volumes/Testing Data/'
segmentation_path = 'numpy_data/Patient Segmentations/Testing Data/'
image_file_list = [image_path + f for f in os.listdir(image_path) if f.endswith('.npy')]
segmentation_file_list = [segmentation_path + f for f in os.listdir(segmentation_path) if f.endswith('.npy')]

image_arrays = []
for file in image_file_list:
    image_array = numpy.load(file)
    image_array.resize(16, 256, 256)
    image_array = numpy.expand_dims(image_array, 3)
    image_arrays.append(image_array)

segmentation_arrays = []
for file in segmentation_file_list:
    segmentation_array = numpy.load(file)
    segmentation_array.resize(16, 256, 256)
    segmentation_array = numpy.expand_dims(segmentation_array, 3)
    segmentation_arrays.append(segmentation_array)

stacked_image_array = numpy.stack(image_arrays)
stacked_segmentation_array = numpy.stack(segmentation_arrays)

print(stacked_image_array.shape)
print(stacked_segmentation_array.shape)

# export as numpy files
image_file = 'numpy_data/Stacked Arrays/test_image_array'
segmentation_file = 'numpy_data/Stacked Arrays/test_segmentation_array'

numpy.save(image_file, stacked_image_array)
numpy.save(segmentation_file, stacked_segmentation_array)

(7, 16, 256, 256, 1)
(7, 16, 256, 256, 1)


In [2]:
%gui qt

In [3]:
from mayavi import mlab

mlab.volume_slice(segmentation_arrays[0])
mlab.volume_slice(image_arrays[0])
mlab.show()